<a href="https://colab.research.google.com/github/AFBA1993/DeepLearningTutorials/blob/main/FFNN_game(RegressionMulticlassPred)/regress%C3%A3o_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
import numpy as np

In [47]:
base = pd.read_csv('/content/drive/MyDrive/UdemyDeepCSVs/games.csv')

In [48]:
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)


In [49]:
base = base.drop('Developer', axis = 1)

In [50]:
base.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,NaN,NaN,NaN,NaN,NaN


In [51]:
base = base.dropna(axis = 0)

In [52]:
base = base.loc[base['NA_Sales'] > 1]
base = base.loc[base['EU_Sales'] > 1]

In [53]:
nome_jogos = base.Name
base = base.drop('Name', axis = 1)

In [54]:
previsores = base.iloc[:, [0,1,2,3,7,8,9,10,11]].values

In [55]:
previsores[:][0]

array(['Wii', 2006.0, 'Sports', 'Nintendo', 76.0, 51.0, '8', 322.0, 'E'],
      dtype=object)

In [56]:
venda_na = base.iloc[:, 4].values
venda_eu = base.iloc[:, 5].values
venda_jp = base.iloc[:, 6].values

In [57]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [58]:
ct = ColumnTransformer(
    [('lol', OneHotEncoder(sparse=False), [0, 2, 3, 8]),],  # the column numbers I want to apply this to
    remainder='passthrough'  # This leaves the rest of my columns in place
)

In [59]:
previsores = ct.fit_transform(previsores)

In [60]:
previsores.shape

(258, 61)

In [61]:
previsores = np.asarray(previsores).astype(np.float32)
venda_na = np.asarray(venda_na).astype(np.float32)
venda_jp = np.asarray(venda_jp).astype(np.float32)
venda_eu = np.asarray(venda_eu).astype(np.float32)

In [62]:
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model

In [63]:
camada_entrada = Input(shape = (61,))

In [64]:
camada_oculta1 = Dense(units = 32, activation = 'sigmoid')(camada_entrada)
camada_oculta2 = Dense(units = 32, activation = 'sigmoid')(camada_oculta1)
camada_saida1 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida2 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida3 = Dense(units = 1, activation = 'linear')(camada_oculta2)

In [65]:
regressor = Model(inputs = camada_entrada,
                  outputs = [camada_saida1, camada_saida2, camada_saida3])

In [66]:
regressor.compile(optimizer = 'adam',
                  loss = 'mse')
regressor.fit(previsores, [venda_na, venda_eu, venda_jp],
              epochs = 5000, batch_size = 100)

A saída de streaming foi truncada nas últimas 5000 linhas.
3/3 [==============================] - 0s 4ms/step - loss: 15.2708 - dense_7_loss: 9.6674 - dense_8_loss: 4.7743 - dense_9_loss: 0.8291
Epoch 2502/5000
3/3 [==============================] - 0s 5ms/step - loss: 15.3101 - dense_7_loss: 9.7066 - dense_8_loss: 4.7817 - dense_9_loss: 0.8218
Epoch 2503/5000
3/3 [==============================] - 0s 6ms/step - loss: 15.3180 - dense_7_loss: 9.7152 - dense_8_loss: 4.7895 - dense_9_loss: 0.8133
Epoch 2504/5000
3/3 [==============================] - 0s 5ms/step - loss: 15.3412 - dense_7_loss: 9.7296 - dense_8_loss: 4.7932 - dense_9_loss: 0.8184
Epoch 2505/5000
3/3 [==============================] - 0s 5ms/step - loss: 15.3346 - dense_7_loss: 9.7136 - dense_8_loss: 4.7927 - dense_9_loss: 0.8283
Epoch 2506/5000
3/3 [==============================] - 0s 4ms/step - loss: 15.3723 - dense_7_loss: 9.7213 - dense_8_loss: 4.8025 - dense_9_loss: 0.8485
Epoch 2507/5000
3/3 [========================

In [67]:
previsao_na, previsao_eu, previsao_jp = regressor.predict(previsores)

In [74]:
previsao_jp[2]

array([2.8041384], dtype=float32)

In [75]:
venda_jp[2]

3.28

In [76]:
from sklearn.metrics import mean_squared_error

In [77]:
mean_squared_error(venda_jp, previsao_jp)

0.8575934

In [78]:
mean_squared_error(venda_eu, previsao_eu)

1.771673

In [79]:
mean_squared_error(venda_na, previsao_na)

4.158457